# Tip5

In [38]:
p = 2**64 -2**32+1
Fp = GF(p)

r = 2**8+1
Fr = GF(r)

## S-boxes

In [39]:
## Define the S-box T.

def T(x):
    """
    Input:
        x:Fp
    Output:
        T(x):Fp
    """
    return x**7

def T_inv(x):
    """
    Inverse of T.
    """
    return x**10540996611094048183

In [40]:
## Define the S-box S.

R = Fp(2**64)

def sigma(x):
    """
    Input:
        x:Fp
    Output:
        x_list:Fr⁸, decomposition of x in base 256
    """
    b = 256
    x_int = ZZ(x)
    x_list = []
    for i in range(8):
        xi = Fr(x_int % b)
        x_list.append(xi)
        x_int //= 256
    return x_list

def L(xi):
    """
    Input:
        xi:Fr
    Output:
        L(xi):Fr
    """
    return (xi+1)**3-1

def L8(x_list):
    """
    Input:
        x_list:Fr⁸
    Output:
        L8(x_list):Fr⁸, L applied elementwise
    """
    x_L8 = [L(xi) for xi in x_list]
    return x_L8

def L_inv(xi):
    """
    Inverse of L.
    """
    return (xi+1)**171-1

def L8_inv(x_list):
    """
    Inverse of L8.
    """
    x_L8_inv = [L_inv(xi) for xi in x_list]
    return x_L8_inv

def rho(x_list):
    """
    Input:
        x_list:Fr⁸
    Output:
        x:Fp, inverse operation of sigma
    """
    x = Fp(0)
    b = 256
    for i in range(8):
        x += Fp(x_list[i])*b**i
    return x
    

def S(x):
    """
    Input:
        x:Fp
    Output:
        S(x):Fp, output of the S-box S
    """
    y = 1/R * rho(L8(sigma(R*x)))
    return y

def S_inv(x):
    """
    Inverse of S.
    """
    y = 1/R * rho(L8_inv(sigma(R*x)))
    return y

## Linear layer

In [41]:
## Linear part of Tip5.

Mcol =  vector(Fp, [
    61402, 1108, 28750, 33823, 7454, 43244, 53865, 12034, 56951, 27521, 41351, 40901, 12021, 59689, 26798, 17845
])
M = matrix.circulant(Mcol).transpose()

## Round permutation and f permutation

In [42]:
## Calcul de la permutation f

def f_round(X, round_constant):
    """
    Input:
        X:Fp^16, state of the permutation
        round_constant:Fp^16, constant added to the round
    Output:
        Y:Fp^16, state after one round
    """
    Y = zero_vector(Fp, 16)
    for i in range(4):
        Y[i] = S(X[i])
    for i in range(4,16):
        Y[i] = T(X[i])
    
    Y = M*Y + round_constant
    
    return Y

def f_round_inv(Y, round_constant):
    """
    Inverse of a round of f.
    """
    M_inv = M.inverse()
    X = M_inv*(Y - round_constant)
    
    for i in range(4):
        X[i] = S_inv(X[i])
    for i in range(4,16):
        X[i] = T_inv(X[i])
    
    return X

def f(X, round_constants):
    """
    Input:
        X:Fp^16, state of the permutation
        round_constant:Fp^16, constant added to the round
    Output:
        Y:Fp^16, state after one round
    """
    Y = X
    for round_constant in round_constants:
        Y = f_round(Y, round_constant)
    
    return Y

## Round constants

In [43]:
## Ajout des round constants

round_constants = [
    vector(Fp, [
        13630775303355457758,
        16896927574093233874,
        10379449653650130495,
        1965408364413093495,
        15232538947090185111,
        15892634398091747074,
        3989134140024871768,
        2851411912127730865,
        8709136439293758776,
        3694858669662939734,
        12692440244315327141,
        10722316166358076749,
        12745429320441639448,
        17932424223723990421,
        7558102534867937463,
        15551047435855531404
            ]
        ),
    vector(Fp, [
        17532528648579384106,
        5216785850422679555,
        15418071332095031847,
        11921929762955146258,
        9738718993677019874,
        3464580399432997147,
        13408434769117164050,
        264428218649616431,
        4436247869008081381,
        4063129435850804221,
        2865073155741120117,
        5749834437609765994,
        6804196764189408435,
        17060469201292988508,
        9475383556737206708,
        12876344085611465020
            ]
        ),
    vector(Fp, [
        13835756199368269249,
        1648753455944344172,
        9836124473569258483,
        12867641597107932229,
        11254152636692960595,
        16550832737139861108,
        11861573970480733262,
        1256660473588673495,
        13879506000676455136,
        10564103842682358721,
        16142842524796397521,
        3287098591948630584,
        685911471061284805,
        5285298776918878023,
        18310953571768047354,
        3142266350630002035
            ]
        ),
    vector(Fp, [
        549990724933663297,
        4901984846118077401,
        11458643033696775769,
        8706785264119212710,
        12521758138015724072,
        11877914062416978196,
        11333318251134523752,
        3933899631278608623,
        16635128972021157924,
        10291337173108950450,
        4142107155024199350,
        16973934533787743537,
        11068111539125175221,
        17546769694830203606,
        5315217744825068993,
        4609594252909613081
            ]
        ),
    vector(Fp, [
        3350107164315270407,
        17715942834299349177,
        9600609149219873996,
        12894357635820003949,
        4597649658040514631,
        7735563950920491847,
        1663379455870887181,
        13889298103638829706,
        7375530351220884434,
        3502022433285269151,
        9231805330431056952,
        9252272755288523725,
        10014268662326746219,
        15565031632950843234,
        1209725273521819323,
        6024642864597845108
            ]
        )
]


## The Tip5 hash function

In [44]:
def Tip5(input_vec):
    """
    Implementation of the 10-to-5 Tip5 hash function.
    
    Input:
        input_vec: Fp^10.
    Output:
        output: Fp^5.
    """
    X = zero_vector(Fp, 16)
    for i in range(10):
        X[i] = input_vec[i]
    for i in range(10,16):
        X[i] = Fp(1)
    
    Y = f(X, round_constants)
    output = Y[:5]
    return output


# Winning one round in univariate cryptanalysis

Find affine spaces such that inverting 2 rounds gives a valid input for Tip5.

Allows to skip 2 rounds in univariate cryptanalysis.

In [ ]:
R = Fp['x0, x1, x2, x3']

def fix_constants(M, round_constant1, round_constant2, target):
    """
    Choose the constant of the affine subspace.
    
    Input:
        M:16x16 matrix, linear layer.
        round_constant1: length 16 vector in Fp, constants of the first round.
        target: the fixed values in the capacity of the hash function.
    Output:
        b: length 16 vector in Fp.
    """
    M_inv = M.inverse()
    target_after_affine_layer = (M_inv * round_constant1)[10:] + target
    M_inv_sub = M_inv[10:, :6]
    constant = M_inv_sub.inverse() * target_after_affine_layer
    b = zero_vector(Fp, 16)
    for i in range(4):
        b[i] = S(constant[i])
    for i in range(4, 6):
        b[i] = T(constant[i])
    b = M*b + round_constant2
    
    return b

def orthogonal_vector(M):
    """
    Input:
        M:16x16 matrix, linear layer.
        i: index of the vector chosen in the basis of the kernel.
    Output:
        a: length 16 vector in Fp.
    """
    M_inv = M.inverse()
    M_inv_sub = M_inv[10:, 6:]
    
    vec = zero_vector(R, 16)
    x = R.gens()
    base = M_inv_sub.right_kernel().basis()
    
    for i in range(4):
        for j in range(16):
            print(j)
            vec[j] += x[i] * base[i][j]
    a = zero_vector(Fp, 16)
    
    for j in range(10):
        a[j+6] = T(vec[j])
    a = M*a
    print(a)
    return a

def control_from_round2(x, a, b):
    """
    Get a valid input from an element in the affine space Fp*a + b.
    """
    y = a*x + b
    
    input_vec = f_round_inv(y, round_constants[1])
    input_vec = f_round_inv(input_vec, round_constants[0])
    return input_vec

# Tests

## Step by step test

In [46]:
## Test couche par couche

input_vector = vector(Fp, range(16))

# Input: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
print("Input vector : ")
print(input_vector)


S_box = zero_vector(Fp, 16)
for i in range(4):
    S_box[i] = S(input_vector[i])
for i in range(4,16):
    S_box[i] = T(input_vector[i])

# Sbox: [0, 1, 8, 27, 
#        16384, 78125, 279936, 823543, 
#        2097152, 4782969, 10000000, 19487171, 
#        35831808, 62748517, 105413504, 170859375]
print("S-box output : ")
print(S_box)

MDS = M*S_box

# MDS: [7205364737005, 12042395183376, 12272828223119, 11340812806600, 
#       17092227296585, 16460982307488, 12559497484731, 17742389185208, 
#       13715383884189, 15300900766312, 14022676374143, 12105605573328, 
#       16882252994553, 11429837802656, 11703409729659, 14542940950688]
print("MDS output : ")
print(MDS)


Input vector : 
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
S-box output : 
(0, 1, 8, 27, 16384, 78125, 279936, 823543, 2097152, 4782969, 10000000, 19487171, 35831808, 62748517, 105413504, 170859375)
MDS output : 
(7205364737005, 12042395183376, 12272828223119, 11340812806600, 17092227296585, 16460982307488, 12559497484731, 17742389185208, 13715383884189, 15300900766312, 14022676374143, 12105605573328, 16882252994553, 11429837802656, 11703409729659, 14542940950688)


## Round by round test

In [47]:
## Test round par round

input_vector = vector(Fp, range(16))

# Input: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
print("Input vector : ")
print(input_vector)

round_vec = f_round(input_vector, round_constants[0])
print("Round 0:")
print(round_vec)

for i in range(1, 5):
    round_vec = f_round(round_vec, round_constants[i])
    print("Round {}:".format(i))
    print(round_vec)
# Round 0, [13630782508720194763, 16896939616488417250, 10379461926478353614, 1965419705225900095, 
#           15232556039317481696, 15892650859074054562, 3989146699522356499, 2851429654516916073, 
#           8709150154677642965, 3694873970563706046, 12692454266991701284, 10722328271963650077, 
#           12745446202694634001, 17932435653561793077, 7558114238277667122, 15551061978796482092]
# Round 1, [5525363112442782795, 3196671190848592148, 6751865969657517721, 6246363638275815365, 
#           17526105136308566870, 6414711837228726335, 17165555941248860828, 10245588381698521362, 
#           17368540314743694648, 14208333065255843385, 12223130565514930992, 6796342552837496564, 
#           3963709218487736004, 11869507492304374432, 7491219276692511038, 17108779980408342790]
# Round 2, [6286105364352512153, 17985860294796311537, 9345400983058519278, 4823584712699818929, 
#           7592681812112444861, 12285819859301081349, 8696939243258074783, 12059135831042518743, 
#           6827571267333842278, 5013808258607437272, 12075345740667337345, 11971779355482478795, 
#           10695815297893868736, 11982168343789428883, 5106944887466967556, 6814426455101604673]
# Round 3, [7125984701079541885, 13703424305218035968, 15382640950439390211, 13892416975492464765, 
#           10013294237979062072, 12926974615054989023, 8765320937915539373, 5249117573702297764, 
#           3064463953341252716, 1630229434993870734, 9905993257711309839, 15255305736124280041, 
#           12189914890971570438, 18428914739834411467, 15996699576317910304, 11053828540506304354]
# Round 4, [14273019456630489802, 12225354657803044645, 18223679466392555512, 4879234115918641111, 
#           198243361942729835, 6697571774370475124, 3935892719377798608, 2781322532457452310, 
#           7475933807446249354, 7334965145562953054, 1275437117587945070, 2445375571864276273, 
#           17005006372293520413, 9537835648539327419, 12703602725074524970, 5428520427373770602]


Input vector : 
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
Round 0:
(13630782508720194763, 16896939616488417250, 10379461926478353614, 1965419705225900095, 15232556039317481696, 15892650859074054562, 3989146699522356499, 2851429654516916073, 8709150154677642965, 3694873970563706046, 12692454266991701284, 10722328271963650077, 12745446202694634001, 17932435653561793077, 7558114238277667122, 15551061978796482092)
Round 1:
(5525363112442782795, 3196671190848592148, 6751865969657517721, 6246363638275815365, 17526105136308566870, 6414711837228726335, 17165555941248860828, 10245588381698521362, 17368540314743694648, 14208333065255843385, 12223130565514930992, 6796342552837496564, 3963709218487736004, 11869507492304374432, 7491219276692511038, 17108779980408342790)
Round 2:
(6286105364352512153, 17985860294796311537, 9345400983058519278, 4823584712699818929, 7592681812112444861, 12285819859301081349, 8696939243258074783, 12059135831042518743, 6827571267333842278, 50138082586074372

## Controlling from round 2

In [65]:
# Choose control vectors
a = orthogonal_vector(M)
b = fix_constants(M, round_constants[0], round_constants[1], vector(Fp, [1,1,1,1,1,1]))

# We get a valid input for Tip5 by choosing an element of an affine subspace as input of round 2.
control_from_round2(Fp(42), a, b)

0
1
2
3
4
5
6
7
8
9
10


IndexError: vector index out of range